In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
HF_TOKEN  = os.getenv("HF_KEY")

In [11]:
llm = ChatOpenAI(
    model="deepseek-ai/DeepSeek-V3-0324",
    api_key = HF_TOKEN,
    base_url="https://router.huggingface.co/v1",
    temperature=0,
    max_tokens = 500,
)

In [4]:
# Define the state structure
class LLMState(TypedDict):
    query: str
    response: str

In [5]:
def llm_query(state: LLMState) -> LLMState:

    query = state['query']
    prompt = f"Answer the following question concisely:\n\n{query}"
    response = llm.invoke(prompt).content
    state['response'] = response
    return state

In [6]:
# Initialize the state graph
graph = StateGraph(LLMState)

# Add notes to the graph
graph.add_node("llm_query", llm_query)
# add edges
graph.add_edge(START, "llm_query")
graph.add_edge("llm_query", END)
# compile the graph
workflow = graph.compile()

In [7]:
# execute the graph
initial_state ={
    "query": "What is the capital of France?",
    "response": ""
}
final_state = workflow.invoke(initial_state)
print(final_state['response'])

The capital of France is **Paris**.  

(Answer is as concise as possible while retaining clarity.)


Prompt Chainning

In [8]:
class BlogState(TypedDict):
    topic: str
    outline: str
    blog_post: str

In [9]:
from prompt_toolkit import prompt


def outline_node(state: BlogState) ->BlogState:
    topic = state['topic']
    prompt = f"Create a detailed outline for a blog post about the following topic: \n{topic}"
    outline = llm.invoke(prompt).content
    state['outline'] = outline
    return state

def blog_post(state : BlogState) ->BlogState:
    outline = state['outline']
    prompt = f"Write a comprehensive blog post based on the following outline: \n{outline}"
    blog_post = llm.invoke(prompt).content
    state['blog_post'] = blog_post
    return state

In [10]:
graph = StateGraph(BlogState)

# Define the nodes
graph.add_node('generate_outline', outline_node)
graph.add_node('write_blog_post', blog_post)
# Define the edges
graph.add_edge(START, 'generate_outline')
graph.add_edge('generate_outline', 'write_blog_post')
graph.add_edge('write_blog_post', END)
# Compile the graph
workFlow = graph.compile()

In [12]:
initial_state = {
    "topic": "The Impact of Artificial Intelligence on Modern Healthcare",
    "outline": "",
    "blog_post": ""
}

final_state = workFlow.invoke(initial_state)
print("Blog Post Outline:\n", final_state['outline'])
print("\nFull Blog Post:\n", final_state['blog_post'])

Blog Post Outline:
 # **Outline: The Impact of Artificial Intelligence on Modern Healthcare**  

## **Introduction**  
- Brief overview of AI and its increasing role in various industries.  
- The growing importance of AI in transforming healthcare.  
- Purpose of the blog post: Explore AI’s applications, benefits, challenges, and future in healthcare.  

---  

## **1. Understanding AI in Healthcare**  
- Definition of AI and its subsets (machine learning, deep learning, natural language processing).  
- How AI differs from traditional computing in healthcare.  
- Historical evolution and milestones of AI in medicine.  

---  

## **2. Key Applications of AI in Healthcare**  

### **A. Enhanced Diagnostics & Early Detection**  
- AI-powered imaging analysis (radiology, pathology, dermatology).  
- Predictive analytics for disease detection (e.g., cancer, diabetic retinopathy).  
- Case studies: IBM Watson, Google DeepMind, Zebra Medical Vision.  

### **B. Precision Medicine & Persona